In [2]:
!pip install optuna

  Obtaining dependency information for optuna from https://files.pythonhosted.org/packages/15/da/68883911855d8b4d521f9a370e4e6aab8232b91c1d8d5a8348c4680c6642/optuna-3.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for alembic>=1.5.0 from https://files.pythonhosted.org/packages/7f/50/9fb3a5c80df6eb6516693270621676980acd6d5a9a7efdbfa273f8d616c7/alembic-1.13.1-py3-none-any.whl.metadata
  Obtaining dependency information for colorlog from https://files.pythonhosted.org/packages/f3/18/3e867ab37a24fdf073c1617b9c7830e06ec270b1ea4694a624038fc40a03/colorlog-6.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for Mako from https://files.pythonhosted.org/packages/c6/c9/9cd84cbd5816aa8bee5fd5a00f857efd636ec30586848d571b67baf0b868/Mako-1.3.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/380.1 kB ? eta -:--:--
   ---- ----------------------------------- 41.0/380.1 kB 2.0 MB/s eta 0:00:01
   ------------ --------------------------- 122.9/38

In [4]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/99.8 MB 1.5 MB/s eta 0:01:06
   ---------------------------------------- 0.1/99.8 MB 1.4 MB/s eta 0:01:10
   ---------------------------------------- 0.2/99.8 MB 1.7 MB/s eta 0:00:58
   ---------------------------------------- 0.3/99.8 MB 1.5 MB/s eta 0:01:05
   ---------------------------------------- 0.3/99.8 MB 1.5 MB/s eta 0:01:05
   ---------------------------------------- 0.3/99.8 MB 1.1 MB/s eta 0:01:29
   ---------------------------------------- 0.3/99.8 MB 1.1 MB/s eta 0:01:31
   ---------------------------------------- 0.4/99.8 MB 998.3 kB/s eta 0:01:40
   ---------------------------------------- 0.4/99.8 MB 998.3 kB/s eta 0:01:40
   -------------------------

In [5]:
import numpy as np
import pandas as pd

import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import optuna
import xgboost as xgb

from sklearn.metrics import r2_score

In [7]:
data = pd.read_csv("C:/Users/iannr/OneDrive/Desktop/Data_Projects/Solar/SolarPrediction.csv")
data

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 12:00:00 AM,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00
32682,1480587301,12/1/2016 12:00:00 AM,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00
32683,1480587001,12/1/2016 12:00:00 AM,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00
32684,1480586702,12/1/2016 12:00:00 AM,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UNIXTime                32686 non-null  int64  
 1   Data                    32686 non-null  object 
 2   Time                    32686 non-null  object 
 3   Radiation               32686 non-null  float64
 4   Temperature             32686 non-null  int64  
 5   Pressure                32686 non-null  float64
 6   Humidity                32686 non-null  int64  
 7   WindDirection(Degrees)  32686 non-null  float64
 8   Speed                   32686 non-null  float64
 9   TimeSunRise             32686 non-null  object 
 10  TimeSunSet              32686 non-null  object 
dtypes: float64(4), int64(3), object(4)
memory usage: 2.7+ MB


In [9]:
print("Total missing values:", data.isna().sum().sum())

Total missing values: 0


In [10]:
#Feature Engineering

data

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 12:00:00 AM,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00
32682,1480587301,12/1/2016 12:00:00 AM,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00
32683,1480587001,12/1/2016 12:00:00 AM,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00
32684,1480586702,12/1/2016 12:00:00 AM,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00


In [13]:
data['Month'] = data['Data'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(int)
data['Day'] = data['Data'].apply(lambda x: re.search(r'(?<=\/)\d+(?=\/)', x).group(0)).astype(int)
data['Year'] = data['Data'].apply(lambda x: re.search(r'(?<=\/)\d+(?=\s)', x).group(0)).astype(int)

data = data.drop('Data', axis=1)

data

,UNIXTime,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet,Month,Day,Year
0,1475229326,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00,9,29,2016
1,1475229023,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00,9,29,2016
2,1475228726,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00,9,29,2016
3,1475228421,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00,9,29,2016
4,1475228124,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00,9,29,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00,12,1,2016
32682,1480587301,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00,12,1,2016
32683,1480587001,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00,12,1,2016
32684,1480586702,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00,12,1,2016


In [14]:
data['Hour'] = data['Time'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(int)
data['Minute'] = data['Time'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(int)
data['Second'] = data['Time'].apply(lambda x: re.search(r'\d+$', x).group(0)).astype(int)

data = data.drop('Time', axis=1)

data

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet,Month,Day,Year,Hour,Minute,Second
0,1475229326,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00,9,29,2016,23,55,26
1,1475229023,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00,9,29,2016,23,50,23
2,1475228726,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00,9,29,2016,23,45,26
3,1475228421,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00,9,29,2016,23,40,21
4,1475228124,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00,9,29,2016,23,35,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00,12,1,2016,0,20,4
32682,1480587301,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00,12,1,2016,0,15,1
32683,1480587001,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00,12,1,2016,0,10,1
32684,1480586702,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00,12,1,2016,0,5,2


In [18]:
print(data.columns)


Index(['UNIXTime', 'Radiation', 'Temperature', 'Pressure', 'Humidity',
       'WindDirection(Degrees)', 'Speed', 'Month', 'Day', 'Year', 'Hour',
       'Minute', 'Second', 'SunriseHour', 'SunriseMinute', 'SunsetHour',
       'SunsetMinute'],
      dtype='object')


In [22]:
data

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,Month,Day,Year,Hour,Minute,Second,SunriseHour,SunriseMinute,SunsetHour,SunsetMinute
0,1475229326,1.21,48,30.46,59,177.39,5.62,9,29,2016,23,55,26,6,13,18,13
1,1475229023,1.21,48,30.46,58,176.78,3.37,9,29,2016,23,50,23,6,13,18,13
2,1475228726,1.23,48,30.46,57,158.75,3.37,9,29,2016,23,45,26,6,13,18,13
3,1475228421,1.21,48,30.46,60,137.71,3.37,9,29,2016,23,40,21,6,13,18,13
4,1475228124,1.17,48,30.46,62,104.95,5.62,9,29,2016,23,35,24,6,13,18,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,1.22,44,30.43,102,145.42,6.75,12,1,2016,0,20,4,6,41,17,42
32682,1480587301,1.17,44,30.42,102,117.78,6.75,12,1,2016,0,15,1,6,41,17,42
32683,1480587001,1.20,44,30.42,102,145.19,9.00,12,1,2016,0,10,1,6,41,17,42
32684,1480586702,1.23,44,30.42,101,164.19,7.87,12,1,2016,0,5,2,6,41,17,42


In [23]:
data['Year'].unique()

array([2016])

In [24]:
data['SunriseHour'].unique()

array([6])

In [25]:
data = data.drop(['Year', 'SunriseHour'], axis=1)

In [26]:
#Splitting and Scaling the data

y = data['Radiation'].copy()
X = data.drop('Radiation', axis=1).copy()

In [27]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=100)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, random_state=200)

In [29]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [34]:
#Hyperparameter Search

def get_model_rmse(params):
    model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'eval')], early_stopping_rounds=10, verbose_eval=0)
    results = model.eval(dval)
    rmse = float(re.search(r'[\d.]+$', results).group(0))
    return rmse

In [35]:
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 0.00001, 10.0)
    max_depth = trial.suggest_int('max_depth', 4, 8)
    l1_reg = trial.suggest_loguniform('l1_reg', 0.00001, 10.0)
    l2_reg = trial.suggest_loguniform('l2_reg', 0.00001, 10.0)
    
    params = {'learning_rate': learning_rate, 'max_depth': max_depth, 'alpha': l1_reg, 'lambda': l2_reg}
    
    return get_model_rmse(params)

In [36]:
study = optuna.create_study()
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-04-23 20:49:42,057] A new study created in memory with name: no-name-51264dd3-0776-43a5-9ab1-9ccdbeb24ef3


  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\iannr\AppData\Local\Temp\ipykernel_10140\647168461.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.00001, 10.0)
C:\Users\iannr\AppData\Local\Temp\ipykernel_10140\647168461.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l1_reg = trial.suggest_loguniform('l1_reg', 0.00001, 10.0)
C:\Users\iannr\AppData\Local\Temp\ipykernel_10140\647168461.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l2_reg = trial.suggest_loguniform('l2_reg', 0.000

[I 2024-04-23 20:49:42,338] Trial 0 finished with value: 88.8145062183527 and parameters: {'learning_rate': 0.1364479732728604, 'max_depth': 7, 'l1_reg': 6.472572395266541e-05, 'l2_reg': 0.002853952265407394}. Best is trial 0 with value: 88.8145062183527.
[I 2024-04-23 20:49:42,665] Trial 1 finished with value: 202.53076575690022 and parameters: {'learning_rate': 0.005588049947079009, 'max_depth': 8, 'l1_reg': 0.035040497174475986, 'l2_reg': 0.0010018171470863794}. Best is trial 0 with value: 88.8145062183527.
[I 2024-04-23 20:49:42,694] Trial 2 finished with value: 105962400.99858712 and parameters: {'learning_rate': 4.200088182288311, 'max_depth': 6, 'l1_reg': 0.08989848402873891, 'l2_reg': 0.005582414666846756}. Best is trial 0 with value: 88.8145062183527.
[I 2024-04-23 20:49:42,810] Trial 3 finished with value: 101.66379053067952 and parameters: {'learning_rate': 0.055124960593955594, 'max_depth': 5, 'l1_reg': 0.0033713817482448724, 'l2_reg': 5.777126168127069}. Best is trial 0 wi

In [37]:
best_params = study.best_params
best_params

{'learning_rate': 0.14078598816095825,
 'max_depth': 8,
 'l1_reg': 0.0002918887709679722,
 'l2_reg': 0.003020362297372515}

In [38]:
model = xgb.train(best_params, dtrain, num_boost_round=10000, evals=[(dval, 'eval')], early_stopping_rounds=10)

[0]	eval-rmse:280.63612
[1]	eval-rmse:247.78955
[2]	eval-rmse:220.20422
[3]	eval-rmse:196.80062
[4]	eval-rmse:177.15583
[5]	eval-rmse:161.26629
[6]	eval-rmse:148.40797
[7]	eval-rmse:137.36449
[8]	eval-rmse:128.54006
[9]	eval-rmse:120.97664
[10]	eval-rmse:114.52377
[11]	eval-rmse:109.39994
[12]	eval-rmse:105.62627
[13]	eval-rmse:102.61335
[14]	eval-rmse:99.94446
[15]	eval-rmse:98.10601
[16]	eval-rmse:96.33698
[17]	eval-rmse:94.75288
[18]	eval-rmse:93.56565
[19]	eval-rmse:92.51768
[20]	eval-rmse:91.54775
[21]	eval-rmse:91.12696
[22]	eval-rmse:90.40922
[23]	eval-rmse:90.22036
[24]	eval-rmse:89.83045
[25]	eval-rmse:89.60515
[26]	eval-rmse:89.22999
[27]	eval-rmse:89.05644
[28]	eval-rmse:88.92493
[29]	eval-rmse:88.72371
[30]	eval-rmse:88.74094
[31]	eval-rmse:88.67437
[32]	eval-rmse:88.54140
[33]	eval-rmse:88.45121
[34]	eval-rmse:88.36853
[35]	eval-rmse:88.31314
[36]	eval-rmse:88.35405
[37]	eval-rmse:88.42624
[38]	eval-rmse:88.29065


C:\Users\iannr\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [20:50:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "l1_reg", "l2_reg" } are not used.

  warnings.warn(smsg, UserWarning)


[39]	eval-rmse:88.17286
[40]	eval-rmse:88.08275
[41]	eval-rmse:88.04430
[42]	eval-rmse:87.93730
[43]	eval-rmse:87.89814
[44]	eval-rmse:87.85623
[45]	eval-rmse:87.66912
[46]	eval-rmse:87.54476
[47]	eval-rmse:87.48002
[48]	eval-rmse:87.48757
[49]	eval-rmse:87.40293
[50]	eval-rmse:87.36135
[51]	eval-rmse:87.30361
[52]	eval-rmse:87.24489
[53]	eval-rmse:87.23639
[54]	eval-rmse:87.27556
[55]	eval-rmse:87.19545
[56]	eval-rmse:87.16136
[57]	eval-rmse:87.14527
[58]	eval-rmse:87.10526
[59]	eval-rmse:87.12864
[60]	eval-rmse:87.16674
[61]	eval-rmse:87.16462
[62]	eval-rmse:87.13520
[63]	eval-rmse:87.21345
[64]	eval-rmse:87.20698
[65]	eval-rmse:87.22125
[66]	eval-rmse:87.21794
[67]	eval-rmse:87.19644


Results

In [40]:
y_true = np.array(y_test, dtype=float)
y_pred = np.array(model.predict(dtest), dtype=float)

In [42]:
r2 = r2_score(y_true, y_pred)

print("R^2 Score: {:.4f}".format(r2))

R^2 Score: 0.9398
